#### Print your name

In [1]:
## Your code here 
print("Exercise by: Janne Bragge")

Exercise by: Janne Bragge


# Introduction to Deep Learning with PyTorch

In this notebook, you'll get introduced to [PyTorch](http://pytorch.org/), a framework for building and training neural networks. PyTorch in a lot of ways behaves like the arrays you love from Numpy. These Numpy arrays, after all, are just tensors. PyTorch takes these tensors and makes it simple to move them to GPUs for the faster processing needed when training neural networks. It also provides a module that automatically calculates gradients (for backpropagation!) and another module specifically for building neural networks. All together, PyTorch ends up being more coherent with Python and the Numpy/Scipy stack compared to TensorFlow and other frameworks.

## Neural Networks

Deep Learning is based on artificial neural networks which have been around in some form since the late 1950s. The networks are built from individual parts approximating neurons, typically called units or simply "neurons." Each unit has some number of weighted inputs. These weighted inputs are summed together (a linear combination) then passed through an activation function to get the unit's output.

<img src="../data/assets/simple_neuron.png" width=400px>

Mathematically this looks like: 

$$
y = f(w_1 x_1 + w_2 x_2 + b) \\
y = f\left(\sum_i w_i x_i +b \right)
$$

With vectors this is the dot/inner product of two vectors:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

## Tensors

It turns out neural network computations are just a bunch of linear algebra operations on *tensors*, a generalization of matrices. A vector is a 1-dimensional tensor, a matrix is a 2-dimensional tensor, an array with three indices is a 3-dimensional tensor (RGB color images for example). The fundamental data structure for neural networks are tensors and PyTorch (as well as pretty much every other deep learning framework) is built around tensors.

<img src="../data/assets/tensor_examples.svg" width=600px>

With the basics covered, it's time to explore how we can use PyTorch to build a simple neural network.

In [2]:
# First, import PyTorch
import torch

## Sigmoid activation function 

Sigmoid function is well known activation function in deep learning:

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/9537e778e229470d85a68ee0b099c08298a1a3f6">

Sigmoid function returns values between [0, 1]:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/640px-Logistic-curve.svg.png" width=200px>


***
### Exercise: Define Sigmoid function. 

***Hint.*** *Use this function frame:*  
```python
def activation_func(x):
  out = your_code_here
  return out
```

**Don't use torch.nn.Sigmoid() -function in this exercise, but code your own solution.** 

In [3]:
## Task 1:
def activation_func(x):
    # Use PyTorch's exp() for element-wise computation
    out = 1 / (1 + torch.exp(-x))  # Sigmoid function formula
    return out

 

In [4]:
### Let's test our awesome sigmoid-function
torch.manual_seed(2) 
x = torch.randn((2, 2))

output = activation_func(x)
print(output)

tensor([[0.5968, 0.4443],
        [0.4208, 0.2306]])


**Tip.** If you did sigmoid -function correctly, you should see the output    
`tensor([[0.5968, 0.4443],
        [0.4208, 0.2306]])`.
***

In [5]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 5 random normal variables
features = torch.randn((1, 5))
# True weights for our data, random normal variables again
weights = torch.randn_like(features)
# and a true bias term
bias = torch.randn((1, 1))

Above I generated data we can use to get the output of our simple network. This is all just random for now, going forward we'll start using normal data. Going through each relevant line:

`features = torch.randn((1, 5))` creates a tensor with shape `(1, 5)`, one row and five columns, that contains values randomly distributed according to the normal distribution with a mean of zero and standard deviation of one. 

`weights = torch.randn_like(features)` creates another tensor with the same shape as `features`, again containing values from a normal distribution.

Finally, `bias = torch.randn((1, 1))` creates a single value from a normal distribution.

## Tensor example

PyTorch tensors can be added, multiplied, subtracted, etc, just like Numpy arrays. In general, you'll use PyTorch tensors pretty much the same way you'd use Numpy arrays. They come with some nice benefits though such as GPU acceleration which we'll get to later. For now, use the generated data to calculate the output of this simple single layer network. 

***
### Exercise 
Calculate the output of the network with input features `features`, weights `weights`, and bias `bias`. Similar to Numpy, PyTorch has a [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum) function, as well as a `.sum()` method on tensors, for taking sums. Use the function `activation_func` defined above as the activation function. 

Set output to variable `output1`.

In [6]:
## Task 2:
# Reuse the existing setup
torch.manual_seed(7)
features = torch.randn((1, 5))  # 1x5 tensor
weights = torch.randn_like(features)  # 1x5 tensor
bias = torch.randn((1, 1))  # 1x1 tensor

# Linear combination: element-wise multiplication, summation, and bias addition
z = torch.sum(features * weights) + bias  # Element-wise product and sum

# Sigmoid activation function
def activation_func(x):
    return 1 / (1 + torch.exp(-x))

output1 = activation_func(z)
 

In [7]:
print(output1)

tensor([[0.1595]])


**Tip.** If you did this correctly, you should see the output    
`tensor([[0.1595]])`.
***

You can do the multiplication and sum in the same operation using a matrix multiplication. In general, you'll want to use matrix multiplications since they are more efficient and accelerated using modern libraries and high-performance computing on GPUs.

Here, we want to do a matrix multiplication of the features and the weights. For this we can use [`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm) or [`torch.matmul()`](https://pytorch.org/docs/stable/torch.html#torch.matmul) which is somewhat more complicated and supports broadcasting. If we try to do it with `features` and `weights` as they are, we'll get an error

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```

As you're building neural networks in any framework, you'll see this often. Really often. What's happening here is our tensors aren't the correct shapes to perform a matrix multiplication. Remember that for matrix multiplications, the number of columns in the first tensor must equal to the number of rows in the second column. Both `features` and `weights` have the same shape, `(1, 5)`. This means we need to change the shape of `weights` to get the matrix multiplication to work.

**Note:** To see the shape of a tensor called `tensor`, use `tensor.shape`. If you're building neural networks, you'll be using this method often.

There are a few options here: [`weights.reshape()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize_()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_), and [`weights.view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view).

* `weights.reshape(a, b)` will return a new tensor with the same data as `weights` with size `(a, b)` sometimes, and sometimes a clone, as in it copies the data to another part of memory.
* `weights.resize_(a, b)` returns the same tensor with a different shape. However, if the new shape results in fewer elements than the original tensor, some elements will be removed from the tensor (but not from memory). If the new shape results in more elements than the original tensor, new elements will be uninitialized in memory. Here I should note that the underscore at the end of the method denotes that this method is performed **in-place**. Here is a great forum thread to [read more about in-place operations](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) in PyTorch.
* `weights.view(a, b)` will return a new tensor with the same data as `weights` with size `(a, b)`.

I usually use `.view()`, but any of the three methods will work for this. So, now we can reshape `weights` to have five rows and one column with something like `weights.view(5, 1)`.

***
### Exercise: 
Calculate the output of our little network using matrix multiplication and activation function `activation_func`.  Set output to variable `output2`.

In [8]:
## Task 3:

# Step 1: Perform matrix multiplication
# Reuse the existing setup
torch.manual_seed(7)  # Set random seed for reproducibility
features = torch.randn((1, 5))  # 1x5 tensor (row vector)
weights = torch.randn((5, 1))  # 5x1 tensor (column vector)
bias = torch.randn((1, 1))  # 1x1 tensor

z = torch.mm(features, weights) + bias  # Matrix multiplication + bias addition

# Step 2: Apply the activation function
output2 = activation_func(z)

 

In [9]:
print(output2)

tensor([[0.1595]])


**Tip.** Should be also   
`tensor([[0.1595]])`.
***

### Stack them up!

That's how you can calculate the output for a single neuron. The real power of this algorithm happens when you start stacking these individual units into layers and stacks of layers, into a network of neurons. The output of one layer of neurons becomes the input for the next layer. With multiple input units and output units, we now need to express the weights as a matrix.

<img src='../data/assets/multilayer_diagram_weights.png' width=450px>

The first layer shown on the bottom here are the inputs, understandably called the **input layer**. The middle layer is called the **hidden layer**, and the final layer (on the right) is the **output layer**. We can express this network mathematically with matrices again and use matrix multiplication to get linear combinations for each unit in one operation. For example, the hidden layer ($h_1$ and $h_2$ here) can be calculated 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

The output for this small network is found by treating the hidden layer as inputs for the output unit. The network output is expressed simply

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

In [10]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 3))

# Define the size of each layer in our network
n_input = features.shape[1]     # Number of input units, must match number of input features
n_hidden = 2                    # Number of hidden units 
n_output = 1                    # Number of output units

# Weights for inputs to hidden layer
W1 = torch.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W2 = torch.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

***
### Exercise: 
Calculate the output for this multi-layer network using the weights `W1` & `W2`, and the biases, `B1` & `B2`. Use activation `activation_func` on both layers. Set output to variable `output3`.

In [11]:
## Task 4:

# Activation function
activation_func = torch.sigmoid

# Compute the hidden layer output
hidden_layer_input = torch.matmul(features, W1) + B1
hidden_layer_output = activation_func(hidden_layer_input)

# Compute the final output layer
output_layer_input = torch.matmul(hidden_layer_output, W2) + B2
output3 = activation_func(output_layer_input)

 

In [12]:
print(output3)

tensor([[0.3171]])


**Tip.** If you did this correctly, you should see the output `tensor([[ 0.3171]])`.
***

The number of hidden units a parameter of the network, often called a **hyperparameter** to differentiate it from the weights and biases parameters. As you'll see later when we discuss training a neural network, the more hidden units a network has, and the more layers, the better able it is to learn from data and make accurate predictions.

## Numpy to Torch and back

Special bonus section! PyTorch has a great feature for converting between Numpy arrays and Torch tensors. To create a tensor from a Numpy array, use `torch.from_numpy()`. To convert a tensor to a Numpy array, use the `.numpy()` method.

In [13]:
import numpy as np
a = np.random.rand(4,3)
a

array([[0.98523946, 0.47340265, 0.08507599],
       [0.49589659, 0.17738327, 0.77342549],
       [0.29218249, 0.86218988, 0.10175151],
       [0.89442249, 0.28067734, 0.44567315]])

In [14]:
b = torch.from_numpy(a)
b

tensor([[0.9852, 0.4734, 0.0851],
        [0.4959, 0.1774, 0.7734],
        [0.2922, 0.8622, 0.1018],
        [0.8944, 0.2807, 0.4457]], dtype=torch.float64)

In [15]:
b.numpy()

array([[0.98523946, 0.47340265, 0.08507599],
       [0.49589659, 0.17738327, 0.77342549],
       [0.29218249, 0.86218988, 0.10175151],
       [0.89442249, 0.28067734, 0.44567315]])

The memory is shared between the Numpy array and Torch tensor, so if you change the values in-place of one object, the other will change as well.

In [16]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)

tensor([[1.9705, 0.9468, 0.1702],
        [0.9918, 0.3548, 1.5469],
        [0.5844, 1.7244, 0.2035],
        [1.7888, 0.5614, 0.8913]], dtype=torch.float64)

In [17]:
# Numpy array matches new values from Tensor
a

array([[1.97047892, 0.94680531, 0.17015197],
       [0.99179317, 0.35476653, 1.54685097],
       [0.58436497, 1.72437976, 0.20350301],
       [1.78884498, 0.56135469, 0.8913463 ]])

## Reflection

Explain briefly following terms (in English or Finnish):
- Deep Learning
- Neural Network
- Activation function
- Tensor
- Pytorch

*Your answers here...*

**DL** = Deep Learning is a subset of Machine Learning focused on algorithms based on artificial neural networks composed of multiple stacked layers. The design of deep learning models is inspired by the biological neural networks of the human brain. It aims to analyze data with a logical structure, mimicking the way humans draw conclusions. **Lo, H. H. & Weng, S. (2020)**

**Neural Network** = A neural network consists of a series of stacked layers. Each layer is made up of units connected to the units in the previous layer through a set of weights. There are various types of layers, but one of the most common is the fully connected layer, where every unit in the layer is directly connected to every unit in the previous layer.
**Foster, D. (2022)**

**Activation Function** = An activation function in machine learning determines whether a neuron in a neural network should be activated or not. It introduces non-linearity to the model, enabling the network to learn complex patterns and relationships in the data. Without activation functions, the network would only be able to model linear relationships, limiting its effectiveness in solving real-world problems. There are many kind of activation function, but three of hte most important are ReLu, sigmoid, and softmax.**Foster, D. (2022)** 

***Tensor*** = A tensor is a multi-dimensional array used in machine learning and deep learning to represent data.
**PyTorch documantation**

***Pytorch*** = PyTorch is an open-source machine learning framework widely used for deep learning. It provides tools to build and train neural networks using dynamic computation graphs, which allow flexibility and debugging during model development. PyTorch uses tensors for efficient data processing and makes it easy to utilize GPUs for accelerated computations
**PyTorch documantation**


------------
***Lo, H. H. & Weng, S. (2020). Machine Learning and Data Science Blueprints for Finance: From Building Trading Strategies to Robo-Advisors Using Python. O'Reilly Media.***

***Foster, D. (2022). Generative Deep Learning (2nd ed.). O'Reilly Media.Foster, D. (2022). Generative Deep Learning (2nd ed.). O'Reilly Media.***

***PyTorch documentation (https://pytorch.org/docs/stable/index.html)***



### Check your answers by running following cell:

In [18]:
# Do not change this code!

import sys
sys.path.insert(0, '../answers/part1/')
from part1_check import *


print("Results:")

correct1 = activation_check(activation_func)

correct2 = part1_check(output1, output2, 
          output3)


print("Correct answers", correct1 + correct2, "/ 4.")



Results:
Correct answers 4 / 4.


### Nice work! 

#### Lähteet
#### PyTorch for Deep Learning & Machine Learning – Full Course - https://www.youtube.com/watch?v=V_xro1bcAuA&t=4272s (0 - 3 tuntia) 
####    + siihen liityvät tehtävät https://github.com/mrdbourke/pytorch-deep-learning/tree/main
####    + kurssimanuaali https://www.learnpytorch.io/


#### Tehtäviä varten perustettu ChatGPT projekti, jossa hyödynnetään PyTorch GPT:tä
#### GPT:llä keskustelen koodista tehtävien ymmärtämiseksi.

